In [1]:
import numpy as np
import pandas as pd
from cvxpy import *
from cvxopt import *
from alphamind.cython.optimizers import QPOptimizer

# Data Preparing
--------------------------

In [2]:
risk_penlty = 0.5

In [3]:
sec_cov_values_full = np.genfromtxt('sec_cov_values.csv', delimiter=',')
signal_full = np.genfromtxt('signal.csv', delimiter=',')

In [4]:
n = 200

sec_cov_values = sec_cov_values_full[:n, :n]
signal = signal_full[:n]

# Optimizing Weights
-------------------------------------

In [5]:
%%time
w = Variable(n)

lbound = 0.
ubound = 1. / n * 20

objective = Minimize(risk_penlty * quad_form(w, sec_cov_values)  - signal * w)
constraints = [w >= lbound,
               w <= ubound,
               sum_entries(w) == 1,]

prob = Problem(objective, constraints)

Wall time: 121 ms


In [6]:
%%time
prob.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.559e-02  -3.082e+01  +3e+03  9e-01  8e-05  1e+00  7e+00    ---    ---    1  1  - |  -  - 
 1  -2.473e+01  -2.541e+01  +7e+01  1e-01  2e-06  1e-02  2e-01  0.9779  1e-04   1  2  2 |  0  0
 2  -1.412e+00  -1.437e+00  +3e+00  5e-03  6e-08  5e-04  7e-03  0.9598  2e-03   2  3  3 |  0  0
 3  -1.019e+00  -1.035e+00  +2e+00  3e-03  4e-08  1e-03  5e-03  0.4692  3e-01   3  4  4 |  0  0
 4  -9.719e-01  -9.842e-01  +2e+00  3e-03  3e-08  2e-03  4e-03  0.5240  6e-01   3  4  3 |  0  0
 5  -3.633e-01  -3.662e-01  +4e-01  7e-04  8e-09  6e-04  1e-03  0.8629  1e-01   3  4  4 |  0  0
 6  -2.058e-01  -2.066e-01  +1e-01  2e-04  2e-09  2e-04  3e-04  0.8197  1e-01   3  4  4 |  0  0
 7  -1.579e-01  -1.581e-01  +3e-02  5e-05  6e-10  5e-05  7e-05  0.8711  1e-01   3  3  3 |  0  0
 8  -1.495e-01  -1.496e-01  +1e-02  2e-05  2e-

-0.14330450116149787

In [7]:
prob.status, prob.value

('optimal', -0.14330450116149787)

In [8]:
%%time
prob.solve(verbose=True, solver='CVXOPT')

     pcost       dcost       gap    pres   dres   k/t
 0: -2.3359e-02 -3.1007e+01  7e+02  1e+01  9e-03  1e+00
 1: -1.1146e+01 -1.5653e+01  6e+01  1e+00  1e-03  2e-02
 2: -1.7778e+00 -2.7042e+00  7e+00  3e-01  3e-04  3e-02
 3: -3.8072e-01 -5.2347e-01  8e-01  5e-02  4e-05  5e-03
 4: -2.0231e-01 -2.3403e-01  2e-01  1e-02  9e-06  1e-03
 5: -1.5540e-01 -1.6212e-01  3e-02  2e-03  2e-06  2e-04
 6: -1.4464e-01 -1.4561e-01  4e-03  3e-04  3e-07  3e-05
 7: -1.4358e-01 -1.4376e-01  8e-04  6e-05  5e-08  4e-06
 8: -1.4340e-01 -1.4346e-01  3e-04  2e-05  2e-08  1e-06
 9: -1.4333e-01 -1.4334e-01  5e-05  4e-06  4e-09  2e-07
10: -1.4331e-01 -1.4331e-01  8e-06  6e-07  5e-10  3e-08
11: -1.4330e-01 -1.4330e-01  4e-07  3e-08  3e-11  1e-09
12: -1.4330e-01 -1.4330e-01  3e-08  2e-09  7e-12  9e-11
Optimal solution found.
Wall time: 195 ms


-0.1433045111926663

In [9]:
prob.status, prob.value

('optimal', -0.1433045111926663)

In [10]:
%%time
P = matrix(sec_cov_values)
q = -matrix(signal)

G = np.zeros((2*n, n))
h = np.zeros(2*n)
for i in range(n):
    G[i, i] = 1.
    h[i] = 1. / n * 20
    G[i+n, i] = -1.
    h[i+n] = 0.
    
G = matrix(G)
h = matrix(h)
    
A = np.ones((1, n))
b = np.ones(1)

A = matrix(A)
b = matrix(b)

sol = solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0: -2.6176e-01 -2.1588e+01  5e+02  2e+01  1e-15
 1: -2.1701e-01 -1.8470e+01  4e+01  8e-01  2e-15
 2: -4.4680e-02 -5.2189e+00  6e+00  5e-02  2e-15
 3: -4.2473e-02 -1.4144e+00  1e+00  1e-02  3e-15
 4: -9.7185e-02 -3.4512e-01  3e-01  1e-03  2e-15
 5: -1.2997e-01 -1.8100e-01  5e-02  1e-04  1e-15
 6: -1.4069e-01 -1.4831e-01  8e-03  1e-05  1e-15
 7: -1.4300e-01 -1.4373e-01  7e-04  9e-07  1e-15
 8: -1.4329e-01 -1.4332e-01  3e-05  2e-08  1e-15
 9: -1.4330e-01 -1.4330e-01  8e-07  3e-10  1e-15
10: -1.4330e-01 -1.4330e-01  5e-08  6e-12  1e-15
Optimal solution found.
Wall time: 55.1 ms


In [16]:
%%time
lbound = np.zeros(n)
ubound = np.ones(n) * 20 / n
cons_matrix = np.ones((1, n))
clb = np.ones(1)
cub = np.ones(1)
qpopt = QPOptimizer(signal, sec_cov_values, lbound, ubound, cons_matrix, clb, cub, 1.)
qpopt.feval()
qpopt.status()

Wall time: 37 ms


# Performace Timing
-------------------------

In [17]:
import datetime as dt

In [18]:
def time_function(py_callable, n):
    start = dt.datetime.now()
    py_callable(n)
    return (dt.datetime.now() - start).total_seconds()

In [19]:
def cvxpy(n):
    w = Variable(n)

    lbound = 0.
    ubound = 0.01

    objective = Minimize(risk_penlty * quad_form(w, sec_cov_values)  - signal * w)
    constraints = [w >= lbound,
                   w <= ubound,
                   sum_entries(w) == 1,]

    prob = Problem(objective, constraints)
    prob.solve(verbose=False, solver='CVXOPT', display=False)

In [20]:
def cvxopt(n):
    P = matrix(sec_cov_values)
    q = -matrix(signal)

    G = np.zeros((2*n, n))
    h = np.zeros(2*n)
    for i in range(n):
        G[i, i] = 1.
        h[i] = 0.01
        G[i+n, i] = -1.
        h[i+n] = 0.

    G = matrix(G)
    h = matrix(h)

    A = np.ones((1, n))
    b = np.ones(1)

    A = matrix(A)
    b = matrix(b)
    
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)

In [23]:
def ipopt(n):
    lbound = np.zeros(n)
    ubound = np.ones(n) * 0.01
    cons_matrix = np.ones((1, n))
    clb = np.ones(1)
    cub = np.ones(1)
    qpopt = QPOptimizer(signal, sec_cov_values, lbound, ubound, cons_matrix, clb, cub, 1.)
    qpopt.feval()

In [24]:
n_steps = list(range(200, 3201, 200))
cvxpy_times = [None] * len(n_steps)
cvxopt_times = [None] * len(n_steps)
ipopt_times = [None] * len(n_steps)
print("{0:<8}{1:>12}{2:>12}{3:>12}".format('Scale(n)', 'cvxpy', 'cvxopt', 'ipopt'))

for i, n in enumerate(n_steps):
    sec_cov_values = sec_cov_values_full[:n, :n]
    signal = signal_full[:n]
    cvxpy_times[i] = time_function(cvxpy, n) * 1000
    cvxopt_times[i] = time_function(cvxopt, n) * 1000
    ipopt_times[i] = time_function(ipopt, n) * 1000
    
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12.2f}".format(n, cvxpy_times[i], cvxopt_times[i], ipopt_times[i]))

Scale(n)       cvxpy      cvxopt       ipopt
200           197.63       45.03       25.02
400          1029.54      392.71       66.05
600          2765.32     1290.11      101.07
800          5906.85     3147.61      123.10
1000        11623.60     5949.30      181.69
1200        19327.03     9837.68      279.76
1400        28894.03    14138.84      529.42
1600        40706.67    21410.84      550.92
1800        59016.47    29610.92      737.62
2000        77980.53    41321.95      818.84
2200        98591.72    52743.93     1310.13
2400       139175.23    76199.42     2156.20
2600       170922.06    89493.43     1591.01
2800       197469.52   112762.10     1569.06
3000       246957.32   135002.46     1484.89
3200       291831.05   160416.66     1919.71


In [21]:
df = pd.DataFrame({'cvxpy': cvxpy_times,
                   'cvxopt': cvxopt_times,
                   'ipopt': ipopt_times},
                  index=n_steps)
df.index.name = 'Problem Scale (n)'
df

,cvxopt,cvxpy,ipopt
Problem Scale (n),,,
200,49.039,207.656,27.019
400,358.899,976.404,51.080
600,1222.190,2811.124,108.764
800,3051.363,5794.853,121.119
1000,5858.366,11435.303,192.262
1200,9806.112,18855.293,272.055
1400,13928.818,28437.877,536.453
1600,20971.899,39959.689,583.744
1800,28056.363,63106.509,630.622
